# RAG Testing

In [1]:
import langchain_community.llms
from langchain_community.llms import Ollama
print(langchain_community.llms.__file__)
from langchain_community.embeddings import OllamaEmbeddings

/Users/tyson/school/cs542-adventure/env/lib/python3.13/site-packages/langchain_community/llms/__init__.py


In [2]:
model = Ollama(model='gpt-oss')
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/ipykernel_74952/2531835591.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  model = Ollama(model='gpt-oss')
/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/ipykernel_74952/2531835591.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")


In [3]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [4]:
from typing import Iterable
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# docs should already have metadata attached. Document objects have a 'metadata' field that is just a dictionary
# (https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document.metadata)
def insert_into_vector_store(docs: Iterable[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,  # chunk size (characters)
        chunk_overlap=20,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(docs)

    document_ids = vector_store.add_documents(documents=all_splits)

    print(f"Split docs into {len(all_splits)} sub-documents.")


In [5]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Remember relevant past events during gameplay in order to inform the next step"""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [7]:
!export LD_LIBRARY_PATH='./env/lib/python3.13/site-packages/jericho'
import jericho
print(jericho.__file__)
import sys
print(sys.path)

GAMES_DIR = "z-machine-games-master/jericho-game-suite"
game = 'zork1.z5'
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

/Users/tyson/school/cs542-adventure/env/lib/python3.13/site-packages/jericho/__init__.py
['/opt/homebrew/Cellar/python@3.13/3.13.11/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.11/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.11/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/tyson/school/cs542-adventure/env/lib/python3.13/site-packages']


OSError: dlopen(/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so, 0x0006): tried: '/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (slice is not valid mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (no such file), '/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (slice is not valid mach-o file), '/private/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (slice is not valid mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/private/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (no such file), '/private/var/folders/zf/p6chjtgj7qdgh66kdctwq36c0000gn/T/tmpe3u_icer/libfrotz.so' (slice is not valid mach-o file)

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    f"Think step by step. You are playing {game}, an interactive fiction game. You must analyze the scenario the game presents to you and choose an action that will make progress. Your goal is to finish the game\n"
    "You have access to a tool that retrieves context from events that have occured in your current playthrough. "
    "Use the tool to help you decide on the next action to take in-game "
)
agent = create_agent(model, tools, system_prompt=prompt)


In [ ]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()
